# AI Project Sub3

In [ ]:
from konlpy.tag import Kkma
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

In [ ]:
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]

# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리

In [ ]:
def load_data(file):
    datas = pd.read_csv(file, skiprows=[0,0], header=None)
    q = datas[0]
    a = datas[1]
    train_q, test_q, train_a, test_a = train_test_split(q, a, test_size=0.35, random_state=321)
    
    return train_q, train_a, test_q, test_a

In [ ]:
# train_q, train_a, test_q, test_a = load_data("./data_in/ChatBotData.csv")

# Req 1-1-2. 텍스트 데이터에 정규화를 사용하여 ([~.,!?\"':;)(]) 제거

In [1]:
def prepro_noise_canceling(sentence):
    RE_FILTER = re.compile("[.,!?\"':;~()]")
    sentence = re.sub(RE_FILTER, "", sentence)
    return sentence

# Req 1-1-3. 텍스트 데이터에 토크나이징

### 알고리즘 순서
    - 텍스트 데이터 prepro_noise_canceling() 함수 처리
    - 띄어쓰기 단위로 나누기
    - 띄어진 단어들로 벡터 형성

In [ ]:
def tokenizing_data(data):
    data = prepro_noise_canceling(data)
    data_splited = list(data.split())
    
    return data_splited

# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리

### 다음 내용들이 적용되어야 함
    - 텍스트 데이터 prepro_noise_canceling()함수 처리
    - 문장을 토큰 단위로 나누기
    - dictionary를 활용하여 토큰 인덱스화
    - dictionary에 없는 토큰의 경우 UNK 값으로 대체
    - 기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기
    - 기준 문장 길이에 맞게 남은 공간이 있다면 pedding 하기
    - 문장 인덱스와 문장 길이 계산

In [1]:
# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리
def enc_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    for val in value:
        val = prepro_noise_canceling(val)
    
        for seq in val:
            # 하나의 seq에 index를 저장할 배열 초기화
            seq_index =[]

            for word in seq.split():
                if dictionary.get(word) is not None:
                    # seq_index에 dictionary 안의 인덱스를 extend 한다
                    seq_index.extend(dictionary[word])
                else:
                    # dictionary에 존재 하지 않는 다면 UNK 값을 extend 한다
                    seq_index.extend(dictionary[UNK])

            # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
            if len(seq_index) > DEFINES.max_sequence_length:
                seq_index = seq_index[:,DEFINES.max_sequence_length]
       
            # seq의 길이를 저장
            seq_len.append(len(seq_index))
            
            # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
            while DEFINES.max_sequence_length < seq_len:
                seq_index += (DEFINES.max_sequence_length - seq_len) * PAD_INDEX

            # 인덱스화 되어 있는 값은 seq_input_index에 추가
            seq_input_index.append(seq_index)
        
    return np.asarray(seq_input_index)